<h1>SAKI SS 2021 Homework {1}</h1>

<h2>Preprocessing and feature extraction.</h2>

In [65]:
import numpy as np
import pandas as pd
import os

input_path = os.path.join("..", "data", "SAKI Exercise 1 - Transaction Classification - Data Set.csv")
df = pd.read_csv(input_path, ';')

for col in df.columns:
    print(col)

Unnamed: 0
Auftragskonto
Buchungstag
Valutadatum
Buchungstext
Verwendungszweck
Beguenstigter/Zahlungspflichtiger
Kontonummer
BLZ
Betrag
Waehrung
label


Each sample consist of 10 feature and one label (without first column which is simply a number of row in dataset). 
<p>Buchungstag/Valutadatum are irrelavent for classification because any transaction can be at any date. Someone can say that the entries of "living" class has rental fee and these payments typically has a date at the end of month (25-31th) but as we see from the table below, there is direct relationship between "Buchungstext" and "Buchungstag": "Meite" has date between 25th-31st and the date for "Lastschrift" or "Euro-Überweisung" is earlier as 25th. So that even for this case, knowledge of the exact date of the transfer will not improve the classifier.<p>

In [66]:
print(df.loc[df['label'] == 'living'][['Buchungstag', 'Buchungstext', 'label']])

    Buchungstag                       Buchungstext   label
1    27.07.2016                              Miete  living
11   28.06.2016                              Miete  living
22   27.05.2016                              Miete  living
29   27.04.2016                              Miete  living
35   29.03.2016                              Miete  living
41   25.02.2016                              Miete  living
65   01.04.2016  Lastschrift (Einzugsermächtigung)  living
76   22.03.2016  Lastschrift (Einzugsermächtigung)  living
83   15.03.2016  Lastschrift (Einzugsermächtigung)  living
84   15.03.2016  Lastschrift (Einzugsermächtigung)  living
94   03.03.2016  Lastschrift (Einzugsermächtigung)  living
101  01.03.2016  Lastschrift (Einzugsermächtigung)  living
117  22.02.2016  Lastschrift (Einzugsermächtigung)  living
118  19.02.2016  Lastschrift (Einzugsermächtigung)  living
123  15.02.2016  Lastschrift (Einzugsermächtigung)  living
124  15.02.2016  Lastschrift (Einzugsermächtigung)  livi

"Auftragskonto" and "Waehrung" do not carry any additional information, because for "Waehrung" there is only one value - EUR, and for "Auftragskonto" there are only two values or it is absent. 

In [67]:
print('Unique values for Waehrung:')
for l in df["Waehrung"].unique():
    print(l)

Unique values for Waehrung:
EUR


In [68]:
print('Unique values for Auftragskonto:')
for l in df["Auftragskonto"].unique():
    print(l)

Unique values for Auftragskonto:
89990201.0
89990210.0
nan


In [69]:
df["Betrag"] = df["Betrag"].apply(str).str.replace(",", ".")
df["Betrag"] = df["Betrag"].astype(float)
df_betrag = pd.DataFrame(columns=['label', 'mean', 'min', 'max'])

for l in df["label"].unique():
    df_betrag = df_betrag.append(
        {'label': l,
        'mean': round(df.loc[df["label"] == l, 'Betrag'].mean()),
        'min': round(df.loc[df["label"] == l, 'Betrag'].min()),
        'max': round(df.loc[df["label"] == l, 'Betrag'].max())}, ignore_index=True
    )

print(df_betrag)   

              label  mean    min   max
0            income  2021   1793  2986
1            living  -286   -670   276
2           private  -161  -1000   100
3  standardOfLiving  -157  -5000    -5
4           leisure   -54   -535     5
5           finance  -358  -4000   200


"Betrag" is also a bad feature because any transaction can has any amount of money. Only "income" is always positive, but even in this case "Buchungstext" is directly related to "Betrag". 

In [70]:
print(df.loc[df['label'] == 'income'][['Betrag', 'Buchungstext', 'label']])

      Betrag   Buchungstext   label
0    2000.00  Lohn / Gehalt  income
10   2000.00  Lohn / Gehalt  income
21   2000.00  Lohn / Gehalt  income
28   2000.00  Lohn / Gehalt  income
34   2000.00  Lohn / Gehalt  income
40   2000.00  Lohn / Gehalt  income
46   2000.00  Lohn / Gehalt  income
48   1792.73   Gehalt/Rente  income
70   1792.73   Gehalt/Rente  income
108  2986.24   Gehalt/Rente  income
147  1792.73   Gehalt/Rente  income
170  2000.00  Lohn / Gehalt  income
181  2000.00  Lohn / Gehalt  income
188  2000.00  Lohn / Gehalt  income
194  2000.00  Lohn / Gehalt  income
200  2000.00  Lohn / Gehalt  income
206  2000.00  Lohn / Gehalt  income


<p>
"Verwendungszweck" - In general, this field can be anything at all, but if you look closely at the table, usually this field contains the same information from other fields, or it is indirectly described using other fields. Having so small database for training, additional transformations or extraction of keywords for clasification will not bring additional advantage, but on the contrary will increase overfitting in relation to new transaction.
<p>
<p>"Beguenstigter/Zahlungspflichtiger" are also not important because "Kontonummer" and "BLZ" are the bank details of those who pay.<p>

In [71]:
df.drop(["Auftragskonto", "Buchungstag", "Verwendungszweck", "Beguenstigter/Zahlungspflichtiger", "Valutadatum", "Betrag", "Waehrung"], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)

for col in df.columns:
    print(col)

Buchungstext
Kontonummer
BLZ
label


<p>For training my classifier i have use only three features: Buchungstext, Kontonummer, BLZ. Categorical variable of features were converted into dummy/indicator variables using pandas.get_dummies.<p>

In [72]:
features = pd.get_dummies(df[df.columns[0:-1]]).values
classes = np.unique(df[df.columns[-1]])
d = dict(zip(classes, np.arange(classes.shape[0])))
labels = df[df.columns[-1]].map(d, na_action='ignore').values

labels = labels.reshape(-1,1)
data = np.hstack((features, labels))

<h2>Classification</h2>

The table shows that there is a class imbalance in this dataset. Applying inappropriate evaluation metrics for model generated using imbalanced data can be dangerous.

In [73]:
print('label               %')
print(df['label'].value_counts()/len(df['label'])*100)

label               %
leisure             31.100478
standardOfLiving    22.488038
finance             15.789474
living              12.440191
private             10.047847
income               8.133971
Name: label, dtype: float64


In this case, other alternative evaluation metrics can be applied such as:
<ol>
  <li>F1 score: harmonic mean of precision and recall.</li>
  <li>MCC: correlation coefficient between the observed and predicted binary classifications.</li>
</ol>

In [82]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef

n = 100
f1 = 0
mcc = 0
for i in range(n):
    gnb = GaussianNB()
    train, test = train_test_split(data, test_size=0.1)
    gnb = gnb.fit(train[:, :-1], train[:, -1])

    y_pred = gnb.predict(test[:, :-1])

    f1 += f1_score(test[:, -1], y_pred, average='micro')
    mcc += matthews_corrcoef(test[:, -1], y_pred)

print("f1  =", round(f1/n, 3))
print("mcc =", round(mcc/n, 3))

f1  = 0.913
mcc = 0.896


After a hundred times the classifier's training with a random splitting of data on the train and test, the average results obtained are quite high. To improve the scores of the classifier, it is necessary to use more data, then perhaps the discarded features can significantly affect the result. 